#**Feature engineering**

In [1]:
# ------------------------------------------
# NOTEBOOK DE CONSOLIDAÇÃO DE FEATURE ENGINEERING
# ------------------------------------------

In [2]:
# ------------------------------------------
# TAXA DE CONVERSAO BRL/USD : 4.40
# ------------------------------------------
PTAX = 4.40

In [3]:
# ------------------------------------------
# modulos pip
# ------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
# ------------------------------------------
# DATABASE: CRIACAO-CONTAS-CHURN-SEM-DUPLICADOS-PROV
# historico dos couriers churneados pela plataforma
#
# SK.CREATED_AT::DATE : data de cadastro do courier na plataforma
# FECHA_ULT : ultima data de acesso do app pelo courier
#
# NOTA: esse database foi recebido com linhas duplicadas
# apos tratamento, total de linhas caiu de 32 milhões para 152,2 mil
# ------------------------------------------
df_churn = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/criacao-contas-churn-sem-duplicados-prov.csv")
df_churn.drop(labels="Unnamed: 0", axis=1, inplace = True)
print(f"Total de linhas: {df_churn.shape[0]}")
df_churn.head(2)

Total de linhas: 152224


,ID,FIRST_NAME,GENDER,CITY,SK.CREATED_AT::DATE,TRANSPORT_MEDIA_TYPE,CARTAO,LEVEL_NAME,FECHA_ULT
0,1286316,Adailton,M,Grande São Paulo,2021-06-07,motorbike,True,bronze,2022-01-16T23:27:35Z
1,1110698,Adriano Floriano Da Silva,M,Recife,2021-02-11,bicycle,True,bronze,2021-07-15T11:16:04Z


In [6]:
# ------------------------------------------
# database contem dados de cidades estrangeiras e nulos
# EXEMPLO : Tuxtla Gutierrez [MX] ; without_city
# ------------------------------------------
df_churn[df_churn['CITY'].isin([ "Tuxtla Gutierrez", "without_city" ])].head()

,ID,FIRST_NAME,GENDER,CITY,SK.CREATED_AT::DATE,TRANSPORT_MEDIA_TYPE,CARTAO,LEVEL_NAME,FECHA_ULT
8727,429842,Marcilon Antônio,M,without_city,2019-10-19,bicycle,True,rookie,2021-09-09T13:48:10Z
10994,1310230,Carla,F,without_city,2021-07-02,motorbike,False,bronze,2021-08-30T15:33:51Z
11318,1127727,Leandro,M,without_city,2021-02-21,bicycle,True,bronze,2021-09-09T13:04:48Z
12060,584838,Wadson,M,Tuxtla Gutierrez,2020-02-27,motorbike,True,bronze,2021-08-06T22:54:12Z
21241,547765,Thyago,M,without_city,2020-01-27,motorbike,True,bronze,2021-09-09T17:50:33Z


In [7]:
# ------------------------------------------
# database contem couriers que abandonaram a plataforma
# entre junho de 2021 a julho de 2022
# ------------------------------------------
df_churn["FECHA_ULT"].map(lambda x : x[:7]).value_counts().sort_index()

2021-06    11633
2021-07    12387
2021-08    11777
2021-09     9838
2021-10     9760
2021-11     8653
2021-12     8374
2022-01     9337
2022-02     9966
2022-03    13781
2022-04    13602
2022-05    15140
2022-06    16030
2022-07     1946
Name: FECHA_ULT, dtype: int64

In [8]:
# ------------------------------------------
# 80% do churn está concentrado nas seguintes cidades
#     . Grande São Paulo
#     . Rio de Janeiro
#     . Belo Horizonte
#     . Recife
#     . Curitiba
#     . Fortaleza
#     . Porto Alegre
#
# DECISAO : trabalhar apenas com couriers que atuem nessas cidades
# ------------------------------------------
df_churn_ = (df_churn["CITY"].value_counts(normalize=True).cumsum() < 0.8).to_frame()
df_churn_ = df_churn_[df_churn_["CITY"] == True]
df_churn_.index

Index(['Grande São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Recife',
       'Curitiba', 'Fortaleza', 'Porto Alegre'],
      dtype='object')

In [9]:
# ------------------------------------------
# CONCLUSAO : filtragem dos couriers que atuam nos principais mercados
# ------------------------------------------
filter = df_churn_.index.to_list()
df_churn__ = df_churn[df_churn['CITY'].isin(filter)]
print(f"Total de linhas: {df_churn__.shape[0]}")
df_churn__.head(2)

Total de linhas: 120206


,ID,FIRST_NAME,GENDER,CITY,SK.CREATED_AT::DATE,TRANSPORT_MEDIA_TYPE,CARTAO,LEVEL_NAME,FECHA_ULT
0,1286316,Adailton,M,Grande São Paulo,2021-06-07,motorbike,True,bronze,2022-01-16T23:27:35Z
1,1110698,Adriano Floriano Da Silva,M,Recife,2021-02-11,bicycle,True,bronze,2021-07-15T11:16:04Z


In [10]:
# ------------------------------------------
# DATABASE : INFOS-GERAIS-PROV
# informacoes gerais de couriers ativos na plataforma
#
# is_active (bool) : true se ativo ; false se inativo
# ------------------------------------------
df_info = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/infos-gerais-prov.csv")
print(f"Total de linhas: {df_info.shape[0]}")
perc_ativos = round(df_info["IS_ACTIVE"].value_counts(normalize=True)[1] * 100, 1)
print(f"% de couriers ativos no database: {perc_ativos}%")
df_info.head(2)

Total de linhas: 180178
% de couriers ativos no database: 99.8%


,ID,NOME,SOBRENOME,GENERO,DATA_NASCIMENTO,CIDADE,IS_ACTIVE,TRANSPORTE,AUTO_ACEITE,COUNT_ORDERS_LAST_7D,...,ULTIMO_PEDIDO,COUNT_ORDERS_RESTAURANTES,COUNT_ORDERS_MERCADO,COUNT_ORDERS_FARMACIA,COUNT_ORDERS_EXPRESS,COUNT_ORDERS_ECOMMERCE,COUNT_ORDERS_ANTOJO,FRETE_MEDIO,COOKING_TIME_MEDIO,ITENS_MEDIO
0,1561246,Wilton Jhonne,Da Silva Abreu,M,1988-04-21,Sao Paulo,True,motorbike,True,1,...,2022-08-01T00:00:00Z,1,0,0,0,0,0,62.255500,10.000000,1.0
1,1561210,Dennis Leonardo,Pereira Santos,M,1998-06-28,Grande São Paulo,True,motorbike,True,7,...,2022-08-01T00:00:00Z,6,1,0,0,0,0,43.444714,23.142857,3.0


In [11]:
# ------------------------------------------
# padronizacao de grafia das cidades usadas para
# filtragem dos couriers
# ------------------------------------------
df_info.replace(
    {"CIDADE": {
        'Sao Paulo': 'Grande São Paulo',
        'São Paulo': 'Grande São Paulo',
        'SÃO PAULO': 'Grande São Paulo',
        'BELO HORIZINTE': 'Belo Horizonte',
        'RIO DE JANEIRO': 'Rio de Janeiro'
    }}, inplace=True
)

In [12]:
# ------------------------------------------
# filtragem dos couriers ativos que atuam nos principais mercados 
# ------------------------------------------
df_info_ = df_info[df_info['CIDADE'].isin(filter)]
df_info_["CIDADE"].unique()

array(['Grande São Paulo', 'Recife', 'Fortaleza', 'Belo Horizonte',
       'Rio de Janeiro', 'Porto Alegre', 'Curitiba'], dtype=object)

In [13]:
# ------------------------------------------
# CONSOLIDACAO DE DATABASES : CHURN *AND* INFOS GERAIS
#
# cruzamento dos couriers inativos de acordo com o database de churn
# e os dados gerais de couriers ativos do database de infos gerais
# 
# RESULTADO ESPERADO : baixa repeticao de IDs entre os databases
# RESULTADO OBSERVADO : alta repeticao de IDs (101.140)
#
# CONCLUSAO : repeticao elevada reflete couriers que abandonam a
# plataforma periodicamente e acabam retornando apos um periodo
# ------------------------------------------
filter = df_churn__["ID"].to_list()
df_info_[df_info_["ID"].isin(filter)].shape[0]

101140

In [14]:
# ------------------------------------------
# EXEMPLO (part 1 de 2):
# id 1544047 - Raquel da Costa Mendes, consta como churn, tendo
# acessado a plataforma pela ultima vez em 03-07-2022
# ------------------------------------------
print(df_churn__[df_churn__["ID"] == 1544047])

            ID FIRST_NAME GENDER            CITY SK.CREATED_AT::DATE  \
44151  1544047     Raquel      F  Belo Horizonte          2022-07-03   

      TRANSPORT_MEDIA_TYPE CARTAO LEVEL_NAME             FECHA_ULT  
44151                  car  False     rookie  2022-07-03T20:29:39Z  


In [15]:
# ------------------------------------------
# EXEMPLO (part 2 de 2):
# id 1544047 - Raquel da Costa Mendes, consta com status ATIVO
# ------------------------------------------
print(df_info_[df_info_["ID"] == 1544047])

           ID    NOME        SOBRENOME GENERO DATA_NASCIMENTO          CIDADE  \
3572  1544047  Raquel  Da Costa Mendes      F      1983-03-01  Belo Horizonte   

      IS_ACTIVE TRANSPORTE  AUTO_ACEITE  COUNT_ORDERS_LAST_7D  ...  \
3572       True        car         True                     0  ...   

             ULTIMO_PEDIDO  COUNT_ORDERS_RESTAURANTES  COUNT_ORDERS_MERCADO  \
3572  2022-07-03T00:00:00Z                          0                     0   

      COUNT_ORDERS_FARMACIA COUNT_ORDERS_EXPRESS COUNT_ORDERS_ECOMMERCE  \
3572                      0                    0                      0   

      COUNT_ORDERS_ANTOJO  FRETE_MEDIO  COOKING_TIME_MEDIO  ITENS_MEDIO  
3572                    0      53.3555                 NaN          2.0  

[1 rows x 25 columns]


In [16]:
# ------------------------------------------
# DECISAO : tierizar os dados de churn (col: IS_ACTIVE)
#     . categoria 1 : True - couriers que nunca foram churneados
#     . categoria 2 : Quasi - couriers inconsistentes
#     . categoria 3 : False - couriers com churn definitivo
# ------------------------------------------

In [17]:
# ------------------------------------------
# filter : lista dos couriers flagados como churn no database CRIACAO CONTAS CHURN
# mask : checa se os couriers flagados constam como ativos no database INFOS GERAIS
# CONCLUSAO : se houver match, o courier é inconsistente e categorizado como Quasi
# ------------------------------------------
filter = df_churn__["ID"].to_list()
mask = df_info_["ID"].isin(filter)
column_name = "IS_ACTIVE"
df_info_.loc[mask, column_name] = "Quasi"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
# ------------------------------------------
# checagem da distribuicao de categorias apos aplicacao do filtro
# ------------------------------------------
df_info_["IS_ACTIVE"].value_counts(normalize=True)

Quasi    0.719806
True     0.279311
False    0.000882
Name: IS_ACTIVE, dtype: float64

In [19]:
# ------------------------------------------
# visualização das dimensões e colunas do database de churn
# ------------------------------------------
print(f"Dimensões do database de churn: {df_churn__.shape}")
df_churn__.columns

Dimensões do database de churn: (120206, 9)


Index(['ID', 'FIRST_NAME', 'GENDER', 'CITY', 'SK.CREATED_AT::DATE',
       'TRANSPORT_MEDIA_TYPE', 'CARTAO', 'LEVEL_NAME', 'FECHA_ULT'],
      dtype='object')

In [20]:
# ------------------------------------------
# visualização das dimensões e colunas do database de infos gerais
# ------------------------------------------
print(f"Dimensões do database de infos gerais: {df_info_.shape}")
df_info_.columns

Dimensões do database de infos gerais: (140510, 25)


Index(['ID', 'NOME', 'SOBRENOME', 'GENERO', 'DATA_NASCIMENTO', 'CIDADE',
       'IS_ACTIVE', 'TRANSPORTE', 'AUTO_ACEITE', 'COUNT_ORDERS_LAST_7D',
       'COUNT_ORDERS_LAST_30D', 'COUNT_ORDERS_CANCELED_LAST_7D',
       'COUNT_ORDERS_CANCELED_LAST_30D', 'GORJETA', 'PRIMEIRO_PEDIDO',
       'ULTIMO_PEDIDO', 'COUNT_ORDERS_RESTAURANTES', 'COUNT_ORDERS_MERCADO',
       'COUNT_ORDERS_FARMACIA', 'COUNT_ORDERS_EXPRESS',
       'COUNT_ORDERS_ECOMMERCE', 'COUNT_ORDERS_ANTOJO', 'FRETE_MEDIO',
       'COOKING_TIME_MEDIO', 'ITENS_MEDIO'],
      dtype='object')

In [21]:
# ------------------------------------------
# filter : lista dos IDs dos couriers ativos nas principais cidades
# mask : checa os couriers que constam exclusivamente da base de churn 
# df_churn__ : dataframe com dados apenas dos couriers que churnearam a
# plataforma de modo definitivo
# ------------------------------------------
filter = df_info_["ID"].to_list()
mask = ~df_churn__["ID"].isin(filter)
df_churn__ = df_churn__[mask]
df_churn__.head(2)

,ID,FIRST_NAME,GENDER,CITY,SK.CREATED_AT::DATE,TRANSPORT_MEDIA_TYPE,CARTAO,LEVEL_NAME,FECHA_ULT
1,1110698,Adriano Floriano Da Silva,M,Recife,2021-02-11,bicycle,True,bronze,2021-07-15T11:16:04Z
2,284886,Bruno,M,Grande São Paulo,2019-07-03,motorbike,False,bronze,2021-07-07T12:33:21Z


In [22]:
# ------------------------------------------
# padronizacao dos nomes das colunas e drop das colunas consideradas
# dispensáveis para a construção do modelo
# ------------------------------------------
df_churn__.rename(columns={
    "FIRST_NAME": "NOME", 
    "GENDER": "GENERO",
    "CITY": "CIDADE",
    "TRANSPORT_MEDIA_TYPE": "TRANSPORTE"
    }, inplace=True)

df_churn__.drop(columns=["SK.CREATED_AT::DATE", "CARTAO", "FECHA_ULT", "LEVEL_NAME"], inplace=True)
df_churn__["IS_ACTIVE"] = False

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [23]:
# ------------------------------------------
# concatenação das bases para consolidação de informações de couriers
# segmentos entre ativos (IS_ACTIVE == True), inativos (IS_ACTIVE == False) e
# inconsistentes (IS_ACTIVE == Quasi)
# ------------------------------------------
df_consol = pd.concat([df_info_, df_churn__])
df_consol.columns

Index(['ID', 'NOME', 'SOBRENOME', 'GENERO', 'DATA_NASCIMENTO', 'CIDADE',
       'IS_ACTIVE', 'TRANSPORTE', 'AUTO_ACEITE', 'COUNT_ORDERS_LAST_7D',
       'COUNT_ORDERS_LAST_30D', 'COUNT_ORDERS_CANCELED_LAST_7D',
       'COUNT_ORDERS_CANCELED_LAST_30D', 'GORJETA', 'PRIMEIRO_PEDIDO',
       'ULTIMO_PEDIDO', 'COUNT_ORDERS_RESTAURANTES', 'COUNT_ORDERS_MERCADO',
       'COUNT_ORDERS_FARMACIA', 'COUNT_ORDERS_EXPRESS',
       'COUNT_ORDERS_ECOMMERCE', 'COUNT_ORDERS_ANTOJO', 'FRETE_MEDIO',
       'COOKING_TIME_MEDIO', 'ITENS_MEDIO'],
      dtype='object')

In [24]:
# ------------------------------------------
# Distribuicao final das categorias de couriers
#
# A partir dessa amostra, poderemos selecionar features e rodar modelos
# com objetivo de melhor categorizar os couriers com base em atributos observaveis
# ------------------------------------------
df_consol["IS_ACTIVE"].value_counts(normalize=True)

Quasi    0.633805
True     0.245939
False    0.120256
Name: IS_ACTIVE, dtype: float64

In [25]:
# ------------------------------------------
# drop de colunas consideradas não relevantes
# ------------------------------------------
df_consol.drop(columns=["NOME", "SOBRENOME"], inplace=True)

In [26]:
# ------------------------------------------
# tratamento e conversão da data de nascimento para numero
# ------------------------------------------
df_consol["DATA_NASCIMENTO"] = df_consol["DATA_NASCIMENTO"].map(lambda x : x[:4], na_action="ignore")
df_consol["DATA_NASCIMENTO"] = pd.to_numeric(df_consol["DATA_NASCIMENTO"])
mean_avg_birthyear = round(df_consol["DATA_NASCIMENTO"].mean(), 0)
df_consol["DATA_NASCIMENTO"].fillna(mean_avg_birthyear, inplace=True)


In [27]:
# ------------------------------------------
# teste de colunas com dados faltantes devido a
# importacao de IDs churneados da base de dados de churn
# ------------------------------------------
if (
    (df_consol[df_consol["COUNT_ORDERS_LAST_7D"].isna()]["IS_ACTIVE"].unique() == False) & 
    (df_consol[df_consol["COUNT_ORDERS_LAST_30D"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_CANCELED_LAST_7D"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_CANCELED_LAST_30D"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["GORJETA"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_RESTAURANTES"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_MERCADO"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_FARMACIA"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_EXPRESS"].isna()]["IS_ACTIVE"].unique() == False) &
    (df_consol[df_consol["COUNT_ORDERS_ANTOJO"].isna()]["IS_ACTIVE"].unique() == False)
    ):
  print("Informações faltantes são apenas de churns definitivos")

Informações faltantes são apenas de churns definitivos


In [28]:
# ------------------------------------------
# confirmada a hipotese acima, os dados podem
# sem completados com 0 sem prejuizo a analise
# ------------------------------------------
df_consol["COUNT_ORDERS_LAST_7D"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_LAST_30D"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_CANCELED_LAST_7D"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_CANCELED_LAST_30D"].fillna(0, inplace=True)
df_consol["GORJETA"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_RESTAURANTES"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_MERCADO"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_FARMACIA"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_EXPRESS"].fillna(0, inplace=True)
df_consol["COUNT_ORDERS_ANTOJO"].fillna(0, inplace=True)

In [29]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 152223
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [30]:
# ------------------------------------------
# DATABASE: DISTANCE-USER-DEF
# ------------------------------------------
df_distance = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/distance-user-def.csv")
print(f"Total de linhas: {df_distance.shape[0]}")
df_distance.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Total de linhas: 31382215


,ORDER_ID,STOREKEEPER_ID,DISTANCE_TO_USER,BUNDLE_ID
0,117818357,NaN,2.315865,NaN
1,147728554,NaN,2.584614,NaN


In [31]:
# ------------------------------------------
# filtragem das linhas em que o ID do courier não foi informado
# ------------------------------------------
df_distance = df_distance[~df_distance["STOREKEEPER_ID"].isna()]

In [32]:
# ------------------------------------------
# agrupa os pedidos por courier e obtem a media de distancia de cada pedido
# ------------------------------------------
df_distance_ = df_distance.groupby(["STOREKEEPER_ID"]).mean()
df_distance_.drop(["ORDER_ID"], axis=1, inplace=True)

In [33]:
# ------------------------------------------
# reseta o index para preparar o dataframe para o merge e troca o nome
# da coluna STOREKEEPER_ID para ID 
# ------------------------------------------
df_distance_ = df_distance_.reset_index()
df_distance_.rename(columns={
    "STOREKEEPER_ID": "ID"
    }, inplace=True)

In [34]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_distance_, how="left", on="ID")
print(f"Dimensoes do dataframe de consolidacao: {df_consol.shape}")
df_consol.info()

Dimensoes do dataframe de consolidacao: (159576, 24)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                  

In [35]:
# ------------------------------------------
# DATABASE: ORDENS-DONE-CANCEL-DEF
# ordens realizadas e ordens canceladas
# Ordens podem ser canceladas pela loja, por falta de produto ou pelo RT
# ------------------------------------------
df_orders = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/ordens-done-cancel-def.csv")
print(f"Total de linhas: {df_orders.shape[0]}")
df_orders.head(2)

Total de linhas: 653166


,STOREKEEPER_ID,ORDERS_DONE,ORDERS_CANCEL,CANCELS_OPS_RT
0,266155,10356,61,0
1,166971,10272,112,4


In [36]:
# ------------------------------------------
# agrupa as ordens canceladas por courier e obtem a soma das ordens completadas,
# canceladas pelo couriers e canceladas pelos time de operacoes
# ------------------------------------------
df_orders_ = df_orders.groupby(["STOREKEEPER_ID"]).sum()

In [37]:
# ------------------------------------------
# reseta o index para preparar o dataframe para o merge e troca o nome
# da coluna STOREKEEPER_ID para ID 
# ------------------------------------------
df_orders_ = df_orders_.reset_index()
df_orders_.rename(columns={
    "STOREKEEPER_ID": "ID"
    }, inplace=True)

In [38]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_orders_, how="left", on="ID")
print(f"Dimensoes do dataframe de consolidacao: {df_consol.shape}")
df_consol.info()

Dimensoes do dataframe de consolidacao: (159576, 27)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                  

In [39]:
# ------------------------------------------
# DATABASE: COMP-DEFECTS-DEF
# sempre que um usuário abre alguma reclamação
# sobre algum pedido incompleto, faltante, item errado,
# é gerada uma ORDER_DEFECT
#
# ORDERS : numero de pedidos que aconteceu algum problema
# GMV_TOTAL : total da transação (GMV = custo total pago pela Rappi para a loja)
# COMPENSATIONS : valor pago (devolvido) para o usuário

# DEFECT_COMPENSATIONS *AND* DEFECT_ORDER : numeros de ordem
# ------------------------------------------
df_defects = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/comp-defects-def.csv")
print(f"Total de linhas: {df_defects.shape[0]}")
df_defects.head(2)

Total de linhas: 6783958


,STOREKEEPER_ID,WEEK,CITY,LEVEL_ID,LEVEL_NAME,ORDERS,GMV_TOTAL,COMPENSATIONS,DEFECT_COMPENSATIONS,DEFECT_ORDER
0,1009854.0,2021-07-17,Grande São Paulo,3.0,bronze,5,422.48,11.173,108019818.0,108019818.0
1,822496.0,2022-01-19,Grande São Paulo,3.0,bronze,5,947.92,1.751,129571770.0,129571770.0


In [40]:
# ------------------------------------------
# padronizacao dos nomes das colunas e drop das colunas consideradas
# dispensáveis para a construção do modelo
#
# agrupa as reclamações por courier e obtem a soma das reclamacoes
# ------------------------------------------
df_defects.drop(columns=["WEEK", "CITY", "LEVEL_ID", "LEVEL_NAME", "DEFECT_COMPENSATIONS", "DEFECT_ORDER"], inplace=True)

df_defects.rename(columns={
    "STOREKEEPER_ID": "ID"
    }, inplace=True)

df_defects_ = df_defects.groupby("ID").sum()

In [41]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_defects_, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [42]:
# ------------------------------------------
# DATABASE: EARNINGS-DEF
# ganhos mensais de cada entregador
# ------------------------------------------
df_earnings = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/earnings-def.csv")
print(f"Total de linhas: {df_earnings.shape[0]}")
df_earnings.head(2)

Total de linhas: 566099


,MONTH,STOREKEEPER_ID,EARNINGS,TIPS
0,2021-07-01T00:00:00Z,33161,1047.82,316.0
1,2021-07-01T00:00:00Z,33189,916.58,304.0


In [43]:
# ------------------------------------------
# padronizacao dos nomes das colunas e drop das colunas consideradas
# dispensáveis para a construção do modelo
#
# agrupa os ganhos por courier por mes e obtem a média desses ganhos
# ------------------------------------------
df_earnings.drop(columns=["MONTH"], inplace=True)

df_earnings.rename(columns={
    "STOREKEEPER_ID": "ID"
    }, inplace=True)

df_earnings = df_earnings.groupby("ID").mean()

In [44]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_earnings, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [45]:
# ------------------------------------------
# DATABASE: TEMPO-RESOLUCAO-MODAL-DEF
# tempo de esperando do suporte tecnico para o courier
# ------------------------------------------
df_resTime = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/tempo-resolucao-modal-def.csv")
print(f"Total de linhas: {df_resTime.shape[0]}")
df_resTime.head(2)

Total de linhas: 1889002


,TICKET_ID,STOREKEEPER_ID,LEVEL_NAME,TRANSPORT_MEDIA_TYPE,SENT_DATA,SENT_HOUR,RESPONSE_AT,RESPONSE_TIME,RESOLUTION_TIME,RESOLUTION_TIME_BUCKET,CITY
0,60e3ba2408e0cf55c8cc8e68,1224698,bronze,bicycle,2021-07-05,23:04:20.917,2021-07-06T02:48:20.283Z,13440,13440.0,(3) Between 2 and 5 hours,Recife
1,61867d939acfd749cfdd3e89,1060149,bronze,car,2021-11-06,10:05:23.781,2021-11-06T10:08:11.863Z,168,87322.0,(6) Between 24 and 72 hours,São José dos Campos


In [46]:
# ------------------------------------------
# padronizacao dos nomes das colunas e drop das colunas consideradas
# dispensáveis para a construção do modelo
# ------------------------------------------
df_resTime = df_resTime.drop(columns=[ 'SENT_DATA', 'SENT_HOUR', 'RESPONSE_AT',
                                                      'CITY' ,'TRANSPORT_MEDIA_TYPE',
                                                      'RESPONSE_TIME', 'RESOLUTION_TIME_BUCKET'])

df_resTime.rename(columns={
    "STOREKEEPER_ID": "ID"
    }, inplace=True)

In [47]:
# ------------------------------------------
# RESOLUTION_TIME : o mesmo ticket de chamado pode aparecer multiplas vezes
# no database a depender da quantidade de interacoes da equipe tecnica até a
# resolução do problema e fechamento do ticket
#
# DECISAO : drop das linhas duplicadas considerando que o RESOLUTION_TIME é
# unico por chamado
# ------------------------------------------
df_resTime = df_resTime.sort_values('RESOLUTION_TIME', ascending=False).drop_duplicates('TICKET_ID').sort_index()

In [48]:
# ------------------------------------------
# transforma medida de tempo de segundos para horas e converta formato do dado
# ------------------------------------------
df_resTime['RESOLUTION_TIME'] = df_resTime['RESOLUTION_TIME'].div(3600)
pd.options.display.float_format = '{:.2f}'.format

In [49]:
# ------------------------------------------
# TEMP_SUM *AND* TEMP_MEAN : agrupa os dados por ID dos couriers e calcula o
# tempo de resolucao total e a media
#
# TOTAL_TICKETS : coluna que representa o numero de vezes que o courier abriu
# um ticket no suporte e dropa os tickets repetidos
#
# DF_RESTIME_ : database final apos a manipulacao e selecao dos dados
# ------------------------------------------
temp_sum = df_resTime.groupby('ID').sum()
temp_sum.rename(columns={'RESOLUTION_TIME': 'RES_TIME_TOTAL'}, inplace = True)

temp_mean = df_resTime.groupby('ID').mean()
temp_mean.rename(columns={'RESOLUTION_TIME': 'RES_TIME_MEAN'}, inplace = True)

df_resTime['TOTAL_TICKETS'] = df_resTime['ID'].map(df_resTime['ID'].value_counts())
df_resTime = df_resTime.drop(columns=['TICKET_ID', 'RESOLUTION_TIME']).drop_duplicates()

df_resTime_ = pd.merge(df_resTime, temp_sum, on= 'ID')
df_resTime_ = pd.merge(df_resTime_, temp_mean, on = 'ID')

In [50]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_resTime_, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [51]:
# ------------------------------------------
# DATABASE: ATTENDANCE-RATE-DEF
# Taxa de aceitação dos pedidos
# Ex: tocou 10 vezes, aceitei 9, 90%
# ------------------------------------------
df_attendance = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/attendance-rate-def.csv")
print(f"Total de linhas: {df_attendance.shape[0]}")
df_attendance.head(2)

Total de linhas: 653167


,STOREKEEPER_ID,ACCEPTANCE_RATE
0,907442.00,0.67
1,1393441.00,0.56


In [52]:
# ------------------------------------------
# 60% dos campos do dataframe não estão preenchidos
# ------------------------------------------
unfilled = df_attendance["ACCEPTANCE_RATE"].isnull().value_counts(normalize=True)
print(f"Campos nao preenchidos: {unfilled}")

Campos nao preenchidos: True    0.60
False   0.40
Name: ACCEPTANCE_RATE, dtype: float64


In [53]:
# ------------------------------------------
# reseta o index para preparar o dataframe para o merge e troca o nome
# da coluna STOREKEEPER_ID para ID 
# ------------------------------------------
df_attendance.rename(columns={
    "STOREKEEPER_ID": "ID"
    }, inplace=True)

In [54]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_attendance, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 37 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [55]:
# ------------------------------------------
# DATABASE: PRODUCT-RETURN-PROV
# Retorno de produto uma vez que a ordem foi cancelada
# Ex: comprei itens de supermercado e por qualquer razão ordem foi cancelada 
# o RT precisa retornar este a loja (nem todas aceitam, ponto ruim) ou devolver 
# em algum ponto de apoio Rappi; Até isso acontecer, ele fica com uma divida
# no valor dos produtos;
# ------------------------------------------
df_productReturn = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/product-return-prov.csv")
print(f"Total de linhas: {df_productReturn.shape[0]}")
df_productReturn.head(2)

Total de linhas: 41535


,ID_ENTREGADOR,LEVEL_NAME,MODAL,CITY,CREATED_AT,ORDER_ID,PRODUCT_RETURNS,VERTICAL_SUB_GROUP,COUNT_TO_GMV,GMV,STORE_ID
0,1522020,rookie,bicycle,Grande São Paulo,2022-06-30T01:20:28-05:00,148999124,-39.71,PHARMACY,False,0.00,900621879
1,625622,diamond,bicycle,Grande São Paulo,2022-06-29T23:16:11-05:00,148997023,-12.46,PHARMACY,True,16.81,900170876


In [56]:
# ------------------------------------------
# dropa as colunas que não serão utilizadas
# renomeia a coluna de ID para possibilitar o merge com df_consol
# calcula media e soma de pedido de retorno de produto por entregador
# ------------------------------------------
df_productReturn.drop([
    'MODAL', 
    'CITY', 
    'STORE_ID', 
    'COUNT_TO_GMV', 
    'GMV', 
    'CREATED_AT', 
    'LEVEL_NAME', 
    'VERTICAL_SUB_GROUP' ], axis=1, inplace=True)

df_productReturn.rename(columns={"ID_ENTREGADOR": "ID"}, inplace = True)

df_temp_mean = df_productReturn.groupby(['ID']).mean().drop(['ORDER_ID'], axis= 1)
df_temp_count = df_productReturn.groupby(['ID']).count().drop(['PRODUCT_RETURNS'], axis=1)
df_productReturn_ = pd.merge(df_temp_mean, df_temp_count, on = 'ID').rename(columns={"ORDER_ID": "N°_PEDIDOS"})
df_productReturn_.head(2)

,PRODUCT_RETURNS,N°_PEDIDOS
ID,,
33051,-0.03,1
33161,-72.44,1


In [57]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_productReturn_, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 39 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [58]:
# ------------------------------------------
# DATABASE: INCIDENTES-REGRAS-RT-DEF
# existem diversas regras para melhoria da qualidade da operação
# exemplo: a regra 92/93 remove/libera o RT do pedido caso ele não esteja em
# movimento ou em direção ao cliente
# ------------------------------------------
df_incidentes = pd.read_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/incidentes-regras-rt-def.csv")
print(f"Total de linhas: {df_incidentes.shape[0]}")
df_incidentes.head(2)

Total de linhas: 2405601


,DATE,NAME,INCIDENT_ID,STOREKEEPER_ID,PUNISHMENT_MINUTES,PUNISHMENT_TYPE,DISCIPLINE_RULE_BUCKET,CATEGORY_RULE,ORDER_ID
0,2022-06-29,92. Liberación (Live),7834134,1089595,15,temporary_block,release_live,Discipline,148929241.00
1,2021-10-11,75. Reporte Manual Casos MUITO GRAVES - Bloque...,6262418,1379883,21600000,permanent_block,Manual,Manual,117857596.00


In [59]:
# ------------------------------------------
# dropa as colunas desnecessarias para o modelo e gera dummy data para
# categorizacao pelo algoritmo de machine learning
# ------------------------------------------
df_incidentes.drop(columns=['ORDER_ID', 'INCIDENT_ID', 'DATE', 'DISCIPLINE_RULE_BUCKET', 'NAME'], inplace=True)
df_incidentes = pd.get_dummies(df_incidentes, columns=['PUNISHMENT_TYPE'])
df_incidentes = pd.get_dummies(df_incidentes, columns=['CATEGORY_RULE'])
df_incidentes.head(2)

,STOREKEEPER_ID,PUNISHMENT_MINUTES,PUNISHMENT_TYPE_permanent_block,PUNISHMENT_TYPE_temporary_block,PUNISHMENT_TYPE_warning,CATEGORY_RULE_Covid,CATEGORY_RULE_Discipline,CATEGORY_RULE_Fraud,CATEGORY_RULE_Manual,CATEGORY_RULE_Other,CATEGORY_RULE_Performance,CATEGORY_RULE_Warning
0,1089595,15,0,1,0,0,1,0,0,0,0,0
1,1379883,21600000,1,0,0,0,0,0,1,0,0,0


In [60]:
# ------------------------------------------
# demonstracao da quantidade de incidencias de cada categoria de punicao
#
# CONCLUSAO : descartar categorias COVID E OTHER
# ------------------------------------------
df_incidentes[[
    'CATEGORY_RULE_Covid', 
    'CATEGORY_RULE_Other', 
    'CATEGORY_RULE_Discipline', 
    'CATEGORY_RULE_Fraud', 
    'CATEGORY_RULE_Manual', 
    'CATEGORY_RULE_Performance', 
    'CATEGORY_RULE_Warning'
    ]].sum()

CATEGORY_RULE_Covid              476
CATEGORY_RULE_Other                4
CATEGORY_RULE_Discipline     2217495
CATEGORY_RULE_Fraud            71962
CATEGORY_RULE_Manual           27200
CATEGORY_RULE_Performance       2902
CATEGORY_RULE_Warning          85562
dtype: int64

In [61]:
# ------------------------------------------
# dropa as colunas desnecessarias para o modelo e padroniza o nome das features
# ------------------------------------------
df_incidentes.drop(columns=['CATEGORY_RULE_Covid', 'CATEGORY_RULE_Other'], inplace=True)
df_incidentes = df_incidentes.rename(columns={
    'STOREKEEPER_ID': 'ID', 
    'PUNISHMENT_TYPE_permanent_block': 'PERMANENT_BLOCK', 
    'PUNISHMENT_TYPE_temporary_block': 'TEMPORARY_BLOCKS', 
    'PUNISHMENT_TYPE_warning': 'WARNINGS', 
    'CATEGORY_RULE_Discipline' : 'DISCIPLINE_INCIDENTS', 
    'CATEGORY_RULE_Fraud' : 'FRAUD_INCIDENTS', 
    'CATEGORY_RULE_Manual' : 'MANUAL_INCIDENTS', 
    'CATEGORY_RULE_Performance' : 'PERFORMANCE_INCIDENTS', 
    'CATEGORY_RULE_Warning' : 'WARNING_INCIDENTS'
    })

In [62]:
# ------------------------------------------
# agrupa os dados por courier e soma a quantidade de incidentes
# ------------------------------------------
df_incidentes = df_incidentes.groupby(by=['ID']).sum().reset_index()

In [63]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_incidentes, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 48 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

In [64]:
# ------------------------------------------
# DATABASE: SUPPLY-DEF
# tempo em Horas que o RT fica/ficou conectado no período
# ------------------------------------------
df_supply = pd.read_csv('/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/new-supply-def.csv')
print(f"Total de linhas: {df_supply.shape[0]}")
df_supply.head(2)

Total de linhas: 7169344


,CITY,STOREKEEPER_ID,VEHICLE_TAG,DATE,HOURS_CONNECTED,ORDERS
0,Fortaleza,937297,car,2022-07-13,5.65,3
1,Rio de Janeiro,1544960,car,2022-07-07,5.40,0


In [65]:
# ------------------------------------------
# cria nova feature derivada do numero de ordens e da quantidade de horas
# ativas do courier - ORDERS_PER_SUPPLY_HOURS
# ------------------------------------------
df_supply.drop(columns=['CITY', 'VEHICLE_TAG', 'DATE'], inplace=True)
df_supply = df_supply.groupby(['STOREKEEPER_ID']).sum().reset_index()
df_supply['ORDERS_PER_HOURS_CONNECTED'] = df_supply['ORDERS'] / df_supply['HOURS_CONNECTED']

In [66]:
# ------------------------------------------
# dropa as colunas desnecessarias para o modelo e padroniza o nome das features
# ------------------------------------------
df_supply.drop(columns=['HOURS_CONNECTED', 'ORDERS'], inplace=True)
df_supply.rename(columns={'STOREKEEPER_ID': 'ID'}, inplace=True)

In [67]:
# ------------------------------------------
# DETALHES SOBRE COMPOSIÇÃO DA BASE CONSOLIDADA
# ------------------------------------------
df_consol = pd.merge(df_consol, df_supply, how="left", on="ID")
df_consol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159576 entries, 0 to 159575
Data columns (total 49 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              159576 non-null  int64  
 1   GENERO                          159576 non-null  object 
 2   DATA_NASCIMENTO                 159576 non-null  float64
 3   CIDADE                          159576 non-null  object 
 4   IS_ACTIVE                       159576 non-null  object 
 5   TRANSPORTE                      159576 non-null  object 
 6   AUTO_ACEITE                     140510 non-null  object 
 7   COUNT_ORDERS_LAST_7D            159576 non-null  float64
 8   COUNT_ORDERS_LAST_30D           159576 non-null  float64
 9   COUNT_ORDERS_CANCELED_LAST_7D   159576 non-null  float64
 10  COUNT_ORDERS_CANCELED_LAST_30D  159576 non-null  float64
 11  GORJETA                         159576 non-null  float64
 12  PRIMEIRO_PEDIDO 

##Limpeza e exportacao dos dados

**Feature: EARNINGS e TIPS**

Para uma melhor precisão da análise do modelo retiramos os valores que não condizem com a realidade dos entregadores em que estamos interessados, decidimos um teto mensal de 2000 dolares como ganho dos pedidos e 1000 dolares como ganho das gorjetas, retirando então todos os ID que não condizem com esses parametros.

In [68]:
# ------------------------------------------
# antes do tramento das features para remocao de outliers
# i.e., empresas que atuam como couriers,
# o dataframe possui informações de 159.576 couriers
# ------------------------------------------
df_consol.shape

(159576, 49)

In [69]:
# ------------------------------------------
# remoção dos outlines da feature EARNINGS
# ------------------------------------------
column_index = df_consol['EARNINGS'][df_consol['EARNINGS'] > 2000].index.tolist()
df_consol.drop(column_index, inplace=True)

In [70]:
# ------------------------------------------
# apos tratamento de EARNINGS, o dataframe passou a ter informacoes
# de 157.637 couriers. Ou seja, cerca de 2.000 empresas prestam serviços de
# courier para a Rappi
#
# também se observa abaixo dados descritivos da amostra
# ------------------------------------------
print(df_consol.shape)
df_consol["EARNINGS"].describe()

(157637, 49)


count   139827.00
mean       242.93
std        353.32
min          0.00
25%         21.31
50%         85.15
75%        311.35
max       1999.87
Name: EARNINGS, dtype: float64

In [71]:
# ------------------------------------------
# remoção dos outlines da feature TIPS
# ------------------------------------------
column_index = df_consol['TIPS'][df_consol['TIPS'] > 1000].index.tolist()
df_consol.drop(column_index, inplace=True)

In [72]:
# ------------------------------------------
# apos tratamento de TIPS, o dataframe passou a ter informacoes
# de 157.627 couriers, apenas 10 couriers a menos do que a amostra anterior
#
# também se observa abaixo dados descritivos da amostra
# ------------------------------------------
print(df_consol.shape)
df_consol["TIPS"].describe()

(157627, 49)


count   139817.00
mean        56.68
std         88.04
min          0.00
25%          4.00
50%         18.00
75%         70.50
max        978.00
Name: TIPS, dtype: float64

In [73]:
# ------------------------------------------
# GERADOR DE CSV PARA MANIPULAÇÃO POR ALGORITMOS
# ------------------------------------------
df_consol.to_csv("/content/drive/Shareddrives/grupo4-rappi-hour/bases-rappi/df-consolidado.csv")